In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
# Keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
import keras.backend as K
from keras.utils.np_utils import to_categorical
# Train-Test
from sklearn.model_selection import train_test_split
# Scaling data
from sklearn.preprocessing import StandardScaler
# Classification Report
from sklearn.metrics import classification_report

In [3]:
df = pd.read_csv('rice.csv')

In [4]:
# Remove Missing Values
na = pd.notnull(df["Status"])
df = df[na]

In [5]:
df = df[["Status", "Soil_Moisture", "Temperature", "Humidity", "Time"]]


In [6]:
from sklearn.preprocessing import StandardScaler
x = df.drop("Status", axis=1)
sc = StandardScaler()
x = pd.DataFrame(sc.fit_transform(x))
y = df["Status"]

In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
    x.values, y, test_size=0.2)

In [8]:
# model = Sequential()
# model.add(Dense(32, input_shape=(4,), activation="relu"))
# model.add(Dense(32, activation="relu"))
# model.add(Dropout(0.5))
# model.add(Dense(1, activation="softmax"))
# model.compile("adam", "binary_crossentropy", metrics=["accuracy"])

inputs = tf.keras.Input(shape=(4,))
x = Dense(32, activation="relu")(inputs)
x = Dense(32, activation="relu")(inputs)
x = Dropout(0.5)(x)
outputs = Dense(1, activation="relu")(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.compile("adam", "binary_crossentropy", metrics=["accuracy"])


In [9]:
model.summary()
model.fit(x_train, y_train, epochs=30, batch_size=100)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 4)]               0         
                                                                 
 dense_1 (Dense)             (None, 32)                160       
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 193
Trainable params: 193
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
800/800 [==============================] - 4s 4ms/step - loss: 1.7053 - accuracy: 0.5928
Epoch 2/30
800/800 [==============================] - 3s 4ms/step - loss: 1.0977 - accuracy: 0.

In [16]:
from sklearn.metrics import mean_squared_error
y_pred = model.predict(x_test)
mean_squared_error(y_test, y_pred)

0.11355762878445608